In [53]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Addtional Imports
import os, time, json
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb

In [54]:
def write_json_file(new_data, filename):
    """Appends a list of records (new_data) to a json file (filename).
    Adapted from: https://www.geeksforgeeks.org/append-to-a-json-file-using-python/"""
    with open(filename, 'r+') as file:
        # First load existing data into a dict.
        file_data = json.load(file)
        ## Choosing to extend or append data
        if (type(new_data)==list) & (type(file_data)==list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # setting file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [55]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    movie = tmdb.Movies(movie_id)
    info = movie.info()

    release = movie.releases()

    for c in release['countries']:
        if c['iso_3166_1'] == 'US':
            info['certification'] = c['certification']
    return info

In [56]:
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['akas_title.csv.gz',
 'basics_title.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'ratings_title.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [57]:
with open(r'C:\Users\zachd\.secret\tmdb_api.json', 'r') as file:
    login = json.load(file)
## Displaying the keys
login.keys()

dict_keys(['api-key'])

In [58]:
tmdb.API_KEY = login['api-key']

In [59]:
# Using the read_csv() method to read the csv file into a dataframe
basics = pd.read_csv("Data/basics_title.csv.gz", low_memory=False)
# Seting the index for YEARS column
YEARS_TO_GET = [2000, 2001]
# Creating empty list to store the errors
errors = []
# Creating a for loop to iterate through the years
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    # Creating a variable to store the json file name
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Checking if the json file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If the file does not exist, create it and add an empty list
    if file_exists == False:
        with open(JSON_FILE, 'w') as file:
            json.dump([{'imdb_id':0}], file)
    # Creating a variable to store the dataframe filtered by the year
    df = basics.loc[basics['startYear']==YEAR].copy()
    # Creating a variable to store the movie ids
    movie_ids = df['tconst'].copy()
    # Loading existing data from json into a daraframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # Filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    # Get index and movie id from list
    for movie_id in tqdm_notebook(movie_ids_to_get, desc=f'Movies from {YEAR}', position=1, leave=True):
        try:
            # Rtrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)
            # Append/extend results to existing file using a pre-made function
            write_json_file(temp, JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.2)
        except Exception as e:
            errors.append([movie_id, e])

    # Read in the json file as a dataframe
    final_yead_df = pd.read_json(JSON_FILE)
    # Save the dataframe as a csv file
    final_yead_df.to_csv(f'{FOLDER}final_tmdb_data_{YEAR}.csv.gz', compression="gzip", index=False)

    print(f'- Number of errors: {len(errors)}')

      

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1225 [00:00<?, ?it/s]

- Number of errors: 210


Movies from 2001:   0%|          | 0/1556 [00:00<?, ?it/s]

- Number of errors: 453


In [60]:
final_2000 = pd.read_csv("Data/final_tmdb_data_2000.csv.gz", low_memory=False)
final_2000.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,22.0,NaN
2,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.100,8.0,NaN
3,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,4.000,1.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.109,2160.0,PG


In [62]:
final_2000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                1228 non-null   object 
 1   adult                  1227 non-null   float64
 2   backdrop_path          671 non-null    object 
 3   belongs_to_collection  112 non-null    object 
 4   budget                 1227 non-null   float64
 5   genres                 1227 non-null   object 
 6   homepage               64 non-null     object 
 7   id                     1227 non-null   float64
 8   original_language      1227 non-null   object 
 9   original_title         1227 non-null   object 
 10  overview               1204 non-null   object 
 11  popularity             1227 non-null   float64
 12  poster_path            1103 non-null   object 
 13  production_companies   1227 non-null   object 
 14  production_countries   1227 non-null   object 
 15  rele

In [61]:
final_2001 = pd.read_csv("Data/final_tmdb_data_2001.csv.gz", low_memory=False)
final_2001.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0035423,0.0,/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,NaN,48000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 14, ...",NaN,11232.0,en,Kate & Leopold,...,76019048.0,118.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"If they lived in the same century, they'd be p...",Kate & Leopold,0.0,6.328,1180.0,PG-13
2,tt0114447,0.0,NaN,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,151007.0,en,The Silent Force,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.000,3.0,NaN
3,tt0118589,0.0,/9NZAirJahVilTiDNCHLFcdkwkiy.jpg,NaN,22000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,10696.0,en,Glitter,...,5271666.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"In music she found her dream, her love, herself.",Glitter,0.0,4.600,122.0,PG-13
4,tt0118652,0.0,/mWxJEFRMvkG4UItYJkRDMgWQ08Y.jpg,NaN,1000000.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",NaN,17140.0,en,The Attic Expeditions,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,His search for peace of mind... will leave his...,The Attic Expeditions,0.0,5.100,29.0,R


In [63]:
final_2001.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1314 entries, 0 to 1313
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                1314 non-null   object 
 1   adult                  1313 non-null   float64
 2   backdrop_path          696 non-null    object 
 3   belongs_to_collection  90 non-null     object 
 4   budget                 1313 non-null   float64
 5   genres                 1313 non-null   object 
 6   homepage               107 non-null    object 
 7   id                     1313 non-null   float64
 8   original_language      1313 non-null   object 
 9   original_title         1313 non-null   object 
 10  overview               1286 non-null   object 
 11  popularity             1313 non-null   float64
 12  poster_path            1179 non-null   object 
 13  production_companies   1313 non-null   object 
 14  production_countries   1313 non-null   object 
 15  rele